https://www.pyimagesearch.com/2019/03/04/holistically-nested-edge-detection-with-opencv-and-deep-learning/

### Download HED pretrained model

In [ ]:
# from urllib.request import urlretrieve

# urlretrieve(HED_MODEL_URL, HED_MODEL_FILEPATH)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import glob
import os

import cv2
from keras.preprocessing.image import load_img
import numpy as np

In [ ]:
HED_MODEL_URL = 'http://vcl.ucsd.edu/hed/hed_pretrained_bsds.caffemodel'
HED_MODEL_FILEPATH = 'hed_model/hed_pretrained_bsds.caffemodel'
HED_MODEL_PROTOTXT_FILEPATH = 'hed_model/deploy.prototxt'

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_N_CHANNELS = 3

USER_COLOR_POINTS_PER_IMG = 50
USER_COLOR_POINTS_CIRCLE_RADIUS = 3

TRAINING_DIR = '../data/training'
TRAINING_TARGET_DIR = f'{TRAINING_DIR}/target'
TRAINING_SOURCE_EDGES_DIR = f'{TRAINING_DIR}/source_edges'
TRAINING_SOURCE_EDGES_COLOR_DIR = f'{TRAINING_DIR}/source_edges+color'
TRAINING_COMPRESSED_DATASET_FILEPATH = f'{TRAINING_DIR}/training_set.npz'

VALIDATION_DIR = '../data/validation'
VALIDATION_TARGET_DIR = f'{VALIDATION_DIR}/target'
VALIDATION_SOURCE_EDGES_DIR = f'{VALIDATION_DIR}/source_edges'
VALIDATION_SOURCE_EDGES_COLOR_DIR = f'{VALIDATION_DIR}/source_edges+color'
VALIDATION_COMPRESSED_DATASET_FILEPATH = f'{VALIDATION_DIR}/validation_set.npz'

In [ ]:
class CropLayer(object):
    def __init__(self, params, blobs):
        # initialize our starting and ending (x, y) coordinates of the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0
        
    def getMemoryShapes(self, inputs):
        # the crop layer will receive two inputs -- we need to crop
        # the first input blob to match the shape of the second one,
        # keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we'll perform the actual
        # crop during the forward pass
        return [[batchSize, numChannels, H, W]]

    def forward(self, inputs):
        # use the derived (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY,
                self.startX:self.endX]]

In [ ]:
def detect_edges(img, hed_model):
    """Edges in image are detected using HED model. 
    
    Returns:
        Grayscale image of detected edges (white background, black foreground).
    """
    blob = cv2.dnn.blobFromImage(
        img, size=(IMG_HEIGHT, IMG_WIDTH), mean=(104.00698793, 116.66876762, 122.67891434))
    hed_model.setInput(blob)
    img_edges = hed_model.forward()
    img_edges = img_edges[0, 0]
    img_edges = (255 * img_edges).astype("uint8")
    img_edges = cv2.bitwise_not(img_edges) # invert black/white
    
    return img_edges

## Load pretrained HED model

In [ ]:
cv2.dnn_registerLayer("Crop", CropLayer)
hed_model = cv2.dnn.readNetFromCaffe(HED_MODEL_PROTOTXT_FILEPATH, HED_MODEL_FILEPATH)

## Generate edge images

In [ ]:
def generate_edge_images(target_dir, source_edges_dir):
    filenames = [os.path.basename(fp) for fp in glob.glob(f'{target_dir}/*.jpg')]
    for idx, filename in enumerate(filenames):
        filepath_target = os.path.join(target_dir, filename)
        img_target = load_img(filepath_target)
        img_target = np.array(img_target)

        img_source_edges = detect_edges(img_target, hed_model)
        img_source_edges_filepath = os.path.join(source_edges_dir, filename)
        cv2.imwrite(img_source_edges_filepath, img_source_edges)

In [ ]:
generate_edge_images(TRAINING_TARGET_DIR, TRAINING_SOURCE_EDGES_DIR)
generate_edge_images(VALIDATION_TARGET_DIR, VALIDATION_SOURCE_EDGES_DIR)

## Compress target and source images into single numpy file

In [ ]:
def compress_dataset(target_dir, source_edges_dir, compressed_dataset_filepath):
    filenames = [os.path.basename(fp) for fp in glob.glob(f'{source_edges_dir}/*.jpg')]
    n_images = len(filenames)
    src_imgs = np.empty((n_images, IMG_HEIGHT, IMG_WIDTH, IMG_N_CHANNELS), dtype=np.uint8)
    target_imgs = np.empty((n_images, IMG_HEIGHT, IMG_WIDTH, IMG_N_CHANNELS), dtype=np.uint8)

    for idx, filename in enumerate(filenames):
        filepath_source_edges = os.path.join(source_edges_dir, filename)
        img_source_edges = load_img(filepath_source_edges)
        src_imgs[idx] = np.array(img_source_edges)

        filepath_target = os.path.join(target_dir, filename)
        img_target = load_img(filepath_target)
        img_target = np.array(img_target)
        target_imgs[idx] = cv2.resize(img_target, (IMG_HEIGHT, IMG_WIDTH))

    print(f'source images: {src_imgs.shape}, target images: {target_imgs.shape}')
    np.savez_compressed(compressed_dataset_filepath, source=src_imgs, target=target_imgs)
    print('save complete')

In [ ]:
compress_dataset(TRAINING_TARGET_DIR, TRAINING_SOURCE_EDGES_DIR, TRAINING_COMPRESSED_DATASET_FILEPATH)
compress_dataset(VALIDATION_TARGET_DIR, VALIDATION_SOURCE_EDGES_DIR, VALIDATION_COMPRESSED_DATASET_FILEPATH)

In [ ]:
def _draw_color_circle_on_src_img(img_src, img_target, center_y, center_x):
    assert(img_src.shape == img_target, "Image source and target must have same shape.")
    
    color = _get_mean_color(img_target, center_y, center_x)
    cv2.circle(img_src, (center_x, center_y), USER_COLOR_POINTS_CIRCLE_RADIUS, color, cv2.FILLED)

def _get_mean_color(img, center_y, center_x):
    radius = USER_COLOR_POINTS_CIRCLE_RADIUS
    h, w = img.shape[:2]
    
    y0 = max(0, center_y-radius)
    y1 = min(h, center_y+radius)
    x0 = max(0, center_x-radius)
    x1 = min(w, center_x+radius)
    mean_color = np.mean(img[y0:y1, x0:x1], axis=(0, 1)).astype(np.uint8)
    
    return mean_color.tolist()
    
def draw_color_circles_on_src_img(img_src, img_target):
    non_white_coords = np.where(~np.all(img_target == 255, axis=2))
    idxs = np.random.choice(len(non_white_coords[0]), USER_COLOR_POINTS_PER_IMG, replace=False)
    for idx in idxs:
        _draw_color_circle_on_src_img(
            img_src, img_target, center_y=non_white_coords[0][idx], center_x=non_white_coords[1][idx])

In [ ]:
filenames = [os.path.basename(fp) for fp in glob.glob(f'{TRAINING_SOURCE_EDGES_DIR}/*.jpg')]
idx = 27
target_fp = os.path.join(TRAINING_TARGET_DIR, filenames[idx])
src_fp = os.path.join(TRAINING_SOURCE_EDGES_DIR, filenames[idx])

In [ ]:
img_target = cv2.imread(target_fp)
img_target = cv2.resize(img_target, (IMG_WIDTH, IMG_HEIGHT))
img_src = cv2.imread(src_fp)
img_src = cv2.resize(img_src, (IMG_WIDTH, IMG_HEIGHT))
draw_color_circles_on_src_img(img_src, img_target)


cv2.imwrite('a.jpg', img_src)
cv2.imwrite('b.jpg', img_target)